# **POSTTEST 2**
**Analisis Dataset**

Pada bagian ini dilakukan analisis awal untuk memahami karakteristik dataset.  
Langkah-langkah analisis meliputi:

**1. Import Library**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline


**2. Load Dataset**

In [ ]:
df = pd.read_csv('top2020.csv')

# Tampilkan 5 record pertama
df.head()


**3. Statistik Deskriptif (numerik)**

In [ ]:
# Menampilkan rata-rata, std, min, max, Q1, Q2, Q3
df.describe()


**4. Tampilkan 10 Record Pertama**

In [ ]:
df.select_dtypes(include='number').head(10)

**5. Informasi Dataset**

In [ ]:
print("Jumlah record:", len(df))
print("Jumlah attribute:", len(df.columns))
print("Nama attribute:", list(df.columns))
print("Jumlah attribute numerik:", len(df.select_dtypes(include='number').columns))

**6. Jumlah Label Unik Utuk Attribute Object**

In [ ]:
categorical_cols = df.select_dtypes(include='object').columns

for col in categorical_cols:
    print(f"Attribute '{col}':")
    print(df[col].value_counts())
    print("-"*30)

**7. Visualisasi Dataset**

Histogram (distribusi numerik)

In [ ]:
num_cols = df.select_dtypes(include='number').columns

df[num_cols].hist(bins=15, figsize=(15,8), color='skyblue', edgecolor='black')
plt.suptitle('Distribusi Attribute Numerik')
plt.show()

Boxplot (untuk outlier)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data=df[num_cols])
plt.title('Boxplot Attribute Numerik')
plt.show()

**8. Korelasi Dataset (heatmap)**

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Heatmap Korelasi Attribute Numerik')
plt.show()

**Kesimpulan analisis**

1. Statistik deskriptif menunjukkan distribusi, rata-rata, standar deviasi, serta nilai min/max/Q1/Q2/Q3 untuk semua attribute numerik.
2. 10 record pertama memberikan gambaran awal data numerik.
3. Informasi dataset mempermudah mengetahui jumlah record, attribute, dan tipe data.
4. Jumlah label unik di attribute kategorikal memberikan insight variabilitas data non-numerik.
5. Histogram & boxplot membantu memahami distribusi dan mendeteksi outlier.
6. Heatmap korelasi memperlihatkan hubungan antar attribute numerik, berguna untuk analisis lebih lanjut.

#**POSTTEST 3**
**Data Preprocessing Dataset**

Pada bagian ini dilakukan preprocessing data agar dataset lebih bersih, terstandarisasi, dan siap dipakai untuk pemodelan machine learning.  
Langkah-langkah yang dilakukan:

**9. Data Cleaning**
   - Menangani missing value.  
   - Menghapus duplicate.  
   - Mengatasi outlier.  


In [ ]:
print("\n--- Missing Value ---")
print("Jumlah missing value sebelum cleaning:")
print(df.isnull().sum())

df = df.dropna()  # drop baris yang ada missing

print("Jumlah missing value setelah cleaning:")
print(df.isnull().sum())

print("\n--- Duplicate Value ---")
print("Jumlah duplikat sebelum cleaning:", df.duplicated().sum())
df = df.drop_duplicates()
print("Jumlah duplikat setelah cleaning:", df.duplicated().sum())

print("\n--- Outlier Handling (IQR Method) ---")
Q1 = df[num_cols].quantile(0.25)
Q3 = df[num_cols].quantile(0.75)
IQR = Q3 - Q1
df = df[~((df[num_cols] < (Q1 - 1.5 * IQR)) | (df[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
print("Jumlah record setelah outlier removal:", len(df))

**10. Normalisasi / Standarisasi Kolom Numerik**
   - Agar skala antar fitur numerik seragam.

In [ ]:
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])
print("\nKolom numerik setelah standarisasi:")
display(df[num_cols].head())

**11. Encoding Kolom Kategorikal**
   - Mengubah data kategorikal menjadi numerik agar bisa digunakan oleh algoritma ML.

In [ ]:
categorical_cols = df.select_dtypes(include='object').columns
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded = pd.DataFrame(
    encoder.fit_transform(df[categorical_cols]),
    columns=encoder.get_feature_names_out(categorical_cols),
    index=df.index
)
df = pd.concat([df.drop(columns=categorical_cols), encoded], axis=1)
print("\nDataset setelah encoding:")
display(df.head())

**12. Feature Engineering**
   - Menambahkan minimal satu fitur baru untuk memperkaya informasi.

In [ ]:
if len(num_cols) >= 2:
    df['feature_ratio'] = df[num_cols[0]] / (df[num_cols[1]] + 1e-9)
    print(f"\nFitur baru 'feature_ratio' ditambahkan dari {num_cols[0]} / {num_cols[1]}")
display(df.head())

**13. Splitting Data**
   - Memisahkan data training & testing untuk evaluasi model.

In [ ]:
X = df.drop(columns=df.columns[-1])
y = df[df.columns[-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\n=== Splitting Data ===")
print("Jumlah data training:", X_train.shape)
print("Jumlah data testing:", X_test.shape)